In [13]:
from constants import AbilityConstantsHandler

In [14]:
ability_constants_handler = AbilityConstantsHandler()
abilities, err = ability_constants_handler.make_query()

In [15]:
unnested_abilities = []
for ability in abilities:
    try:
        ability['displayName'] = ability['language']['displayName']
        ability['description'] = ability['language']['description']
        del ability['language']
    except:
        ability['displayName'] = None
        ability['description'] = None
    unnested_abilities.append(ability)
    

In [26]:
from sql_client import SQLClient
sqlclient = SQLClient()
cur = sqlclient.cur

In [28]:
for ability in unnested_abilities:
    try:
        cur.execute("""INSERT INTO abilityConstants (id, name, abilityName, displayName, description) VALUES (%s, %s, %s, %s, %s)""",
        (ability['id'], ability['name'], ability['abilityName'], ability['displayName'], ability['description']))
    except:
        print(f'Error inserting {ability["id"]}')

sqlclient.conn.commit()
sqlclient.close_connection()